# Editing data files for SQL database

The source files contain so much information that their size is really large. Therefore parts extract from the files and store into the database tables.

After the coordinate format has been converted, the source files are split into three parts:
1. Onnettomuudet (target file)
2. Alueet (result file)
3. Tieominaisuudet (result2 file)

Data has dependence on each other, which is defined by key values when sharing files. After splitting and creating new files with key values, new files store waiting for inserting SQL Server database. Inserting files into the database is done with a different code.

### Define parameters of the source and result files

In [1]:
# File path and name
year = 2014
src_file = '../datasets/onnettomuus/co/tielonnett_'+str(year)+'_onnett.csv'
target_file = '../datasets/onnettomuus/db/tielonnett_'+str(year)+'_onnett.csv'
result_file = '../datasets/onnettomuus/db/tielonnett_'+str(year)+'_alue.csv'
result2_file = '../datasets/onnettomuus/db/tielonnett_'+str(year)+'_tieomin.csv'

# File attributes
src_delimiter = ';'
src_encoding = 'utf-8'
rslt_delimiter = ';'
rslt_encoding = 'utf-8'

# Needed new id column names
handle_frame_id = 'alue_id'
handle2_frame_id = 'tieomin_id'

In [2]:
# Used dataframe names
# source_dataframe = 'df'
# handle_dataframe = 'df_a' -> result_dataframe = 'df_r'
# handle2_dataframe = 'df_b' -> result2_dataframe = 'df_r2'
# src_dropped_df = 'df_as' -> src_handled_df = 'df1'
# src_dropped2_df = 'df_bs' -> src_handled2_df = 'df2'
# target_dataframe = 'df_t' (combined df1 and df2)

In [3]:
# Used dataframe column lists
# Use in result file and Drop from target file
hndl_file_cols = ['ELY','Elynimi','Poliisipri','Piirinimi','Maakunta','Maakuntsel','Kunta','Kuntasel']
# Use in result2 file
hndl2_file_cols = ['Vuosi','Tienpit','Tienpitsel','Tie','Aosa','Aet','Ajr','Taajmerk','Taajamasel','Liikvalot',
                   'Liikvalsel','Liittyvtie','Noplaji','Noplajisel','Nopsuunvas','Nopsuunoik','Taajama','Mo_mol',
                   'Mo_molsel','Toimluokka','Toimlksel','Kvl','Raskaskvl','Tienlev','Oslakpvm','Tietyö',
                   'Päällyste','Päällsel','Risteys','Risteyssel','Rautatie','Rautatsel','Muuliit','Muuliitsel',
                   'Tietyyppi','Tietyypsel','Talvhoitlk','Talvhoitsel','Tienverkas','Tienverkse','Maankäyttö',
                   'Maankäytse','Valoohjaus','Valoohjsel','Lisäkaisty','Lisäkaisse','Solmutyyp','Solmutyyps',
                   'Liitluok','Liitlksel','Lähliittie','Suuntlkm','Toimenpide','Toimpidsel','Luovpvm','Valaisomis',
                   'Valomsel','Poikkileik','Poikleikse','Päällyslev','Päällystlk','Päällksel',
                   'Nakos150','Nakos300','Nakos460','Runkotie','Raskos']
# Drop from target file
hndl2_onnett_cols = ['Tienpit','Tienpitsel','Tie','Aosa','Aet','Ajr','Taajmerk','Taajamasel','Liikvalot',
                   'Liikvalsel','Liittyvtie','Noplaji','Noplajisel','Nopsuunvas','Nopsuunoik','Taajama','Mo_mol',
                   'Mo_molsel','Toimluokka','Toimlksel','Kvl','Raskaskvl','Tienlev','Oslakpvm','Tietyö',
                   'Päällyste','Päällsel','Risteys','Risteyssel','Rautatie','Rautatsel','Muuliit','Muuliitsel',
                   'Tietyyppi','Tietyypsel','Talvhoitlk','Talvhoitsel','Tienverkas','Tienverkse','Maankäyttö',
                   'Maankäytse','Valoohjaus','Valoohjsel','Lisäkaisty','Lisäkaisse','Solmutyyp','Solmutyyps',
                   'Liitluok','Liitlksel','Lähliittie','Suuntlkm','Toimenpide','Toimpidsel','Luovpvm','Valaisomis',
                   'Valomsel','Poikkileik','Poikleikse','Päällyslev','Päällystlk','Päällksel',
                   'Nakos150','Nakos300','Nakos460','Runkotie','Raskos']

In [4]:
# Used libraries
import pandas as pd
import numpy as np
import math

### Download source file

In [5]:
df = pd.read_csv(src_file, delimiter=src_delimiter, encoding=src_encoding)

C:\Users\Pasi\AppData\Local\Temp\ipykernel_7668\842088147.py:1: DtypeWarning: Columns (55,87) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(src_file, delimiter=src_delimiter, encoding=src_encoding)


In [6]:
df

,Onnett_id,Tienpit,Tienpitsel,Tie,Aosa,Aet,Ajr,Vuosi,Kk,Päivä,...,Päällyslev,Päällystlk,Päällksel,Nakos150,Nakos300,Nakos460,Runkotie,Raskos,position.lat,position.lon
0,6761989,1,Liikennevirasto,1.0,3.0,3.0,2.0,2014,10,30.10.2014,...,114.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei,60.202561,24.881814
1,6652962,1,Liikennevirasto,1.0,3.0,8.0,1.0,2014,1,07.01.2014,...,104.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,on,60.204074,24.880904
2,6840693,1,Liikennevirasto,1.0,3.0,8.0,1.0,2014,8,29.08.2014,...,104.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei,60.204190,24.880878
3,6682862,1,Liikennevirasto,1.0,3.0,21.0,2.0,2014,3,15.03.2014,...,114.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei,60.202459,24.881586
4,6646346,1,Liikennevirasto,1.0,3.0,30.0,2.0,2014,11,15.11.2014,...,115.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,on,60.202429,24.881425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28340,6924514,3,Muu,NaN,NaN,NaN,NaN,2014,8,07.08.2014,...,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,on,60.228066,25.064978
28341,6924567,3,Muu,NaN,NaN,NaN,NaN,2014,7,25.07.2014,...,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei,60.861603,26.667586
28342,6924574,3,Muu,NaN,NaN,NaN,NaN,2014,8,06.08.2014,...,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei,61.492871,23.496168
28343,6924769,3,Muu,NaN,NaN,NaN,NaN,2014,7,26.07.2014,...,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei,60.401336,22.366238


#### Check column names of the file

In [7]:
#list(df.columns)

## A. Editing first result file

#### A.1 Split area columns to new handle dataframe

In [8]:
df_a = pd.DataFrame(df[hndl_file_cols])
df_a = df_a.drop_duplicates()

In [9]:
df_a

,ELY,Elynimi,Poliisipri,Piirinimi,Maakunta,Maakuntsel,Kunta,Kuntasel
0,1,Uusimaa,0,Ei arvoa,1,Uusimaa,91,Helsinki
16,1,Uusimaa,0,Ei arvoa,1,Uusimaa,49,Espoo
44,1,Uusimaa,0,Ei arvoa,1,Uusimaa,257,Kirkkonummi
54,1,Uusimaa,0,Ei arvoa,1,Uusimaa,927,Vihti
71,1,Uusimaa,0,Ei arvoa,1,Uusimaa,444,Lohja
...,...,...,...,...,...,...,...,...
18927,4,Pirkanmaa,0,Ei arvoa,6,Pirkanmaa,604,Pirkkala
19142,10,Etelä-Pohjanmaa,0,Ei arvoa,14,Etelä-Pohjanmaa,164,Jalasjärvi
22422,2,Varsinais-Suomi,0,Ei arvoa,4,Satakunta,319,Köyliö
27392,1,Uusimaa,0,Ei arvoa,7,Päijät-Häme,283,Hämeenkoski


#### A.2 Create new id value to new handle dataframe

In [10]:
# Set new id value column to last
df_a[handle_frame_id] = df_a['ELY'].map(str) + df_a['Poliisipri'].map(str) + df_a['Maakunta'].map(str) + df_a['Kunta'].map(str)
df_a[[handle_frame_id]] = df_a[[handle_frame_id]].astype('int64')
df_a

,ELY,Elynimi,Poliisipri,Piirinimi,Maakunta,Maakuntsel,Kunta,Kuntasel,alue_id
0,1,Uusimaa,0,Ei arvoa,1,Uusimaa,91,Helsinki,10191
16,1,Uusimaa,0,Ei arvoa,1,Uusimaa,49,Espoo,10149
44,1,Uusimaa,0,Ei arvoa,1,Uusimaa,257,Kirkkonummi,101257
54,1,Uusimaa,0,Ei arvoa,1,Uusimaa,927,Vihti,101927
71,1,Uusimaa,0,Ei arvoa,1,Uusimaa,444,Lohja,101444
...,...,...,...,...,...,...,...,...,...
18927,4,Pirkanmaa,0,Ei arvoa,6,Pirkanmaa,604,Pirkkala,406604
19142,10,Etelä-Pohjanmaa,0,Ei arvoa,14,Etelä-Pohjanmaa,164,Jalasjärvi,10014164
22422,2,Varsinais-Suomi,0,Ei arvoa,4,Satakunta,319,Köyliö,204319
27392,1,Uusimaa,0,Ei arvoa,7,Päijät-Häme,283,Hämeenkoski,107283


#### A.3 Create new column with values from old row index value to new handle dataframe

In [11]:
# Place new index value column to first
df_a = df_a.rename_axis('index_val').reset_index()

In [12]:
idx = df_a.index
df_a.loc[idx]

,index_val,ELY,Elynimi,Poliisipri,Piirinimi,Maakunta,Maakuntsel,Kunta,Kuntasel,alue_id
0,0,1,Uusimaa,0,Ei arvoa,1,Uusimaa,91,Helsinki,10191
1,16,1,Uusimaa,0,Ei arvoa,1,Uusimaa,49,Espoo,10149
2,44,1,Uusimaa,0,Ei arvoa,1,Uusimaa,257,Kirkkonummi,101257
3,54,1,Uusimaa,0,Ei arvoa,1,Uusimaa,927,Vihti,101927
4,71,1,Uusimaa,0,Ei arvoa,1,Uusimaa,444,Lohja,101444
...,...,...,...,...,...,...,...,...,...,...
308,18927,4,Pirkanmaa,0,Ei arvoa,6,Pirkanmaa,604,Pirkkala,406604
309,19142,10,Etelä-Pohjanmaa,0,Ei arvoa,14,Etelä-Pohjanmaa,164,Jalasjärvi,10014164
310,22422,2,Varsinais-Suomi,0,Ei arvoa,4,Satakunta,319,Köyliö,204319
311,27392,1,Uusimaa,0,Ei arvoa,7,Päijät-Häme,283,Hämeenkoski,107283


In [13]:
# Get column place number of new id value
cn = df_a.columns.get_loc(handle_frame_id)
cn

9

#### A.4 Create new column with values from old row index value to source dataframe

In [14]:
# Create new dataframe without dropped columns
df_as = df.drop(columns=hndl_file_cols)

In [15]:
df_as

,Onnett_id,Tienpit,Tienpitsel,Tie,Aosa,Aet,Ajr,Vuosi,Kk,Päivä,...,Päällyslev,Päällystlk,Päällksel,Nakos150,Nakos300,Nakos460,Runkotie,Raskos,position.lat,position.lon
0,6761989,1,Liikennevirasto,1.0,3.0,3.0,2.0,2014,10,30.10.2014,...,114.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei,60.202561,24.881814
1,6652962,1,Liikennevirasto,1.0,3.0,8.0,1.0,2014,1,07.01.2014,...,104.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,on,60.204074,24.880904
2,6840693,1,Liikennevirasto,1.0,3.0,8.0,1.0,2014,8,29.08.2014,...,104.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei,60.204190,24.880878
3,6682862,1,Liikennevirasto,1.0,3.0,21.0,2.0,2014,3,15.03.2014,...,114.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei,60.202459,24.881586
4,6646346,1,Liikennevirasto,1.0,3.0,30.0,2.0,2014,11,15.11.2014,...,115.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,on,60.202429,24.881425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28340,6924514,3,Muu,NaN,NaN,NaN,NaN,2014,8,07.08.2014,...,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,on,60.228066,25.064978
28341,6924567,3,Muu,NaN,NaN,NaN,NaN,2014,7,25.07.2014,...,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei,60.861603,26.667586
28342,6924574,3,Muu,NaN,NaN,NaN,NaN,2014,8,06.08.2014,...,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei,61.492871,23.496168
28343,6924769,3,Muu,NaN,NaN,NaN,NaN,2014,7,26.07.2014,...,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei,60.401336,22.366238


In [16]:
# Place new index value column to first
df_as = df_as.rename_axis('index_val').reset_index()

In [17]:
idx = df_as.index
df_as.loc[idx]

,index_val,Onnett_id,Tienpit,Tienpitsel,Tie,Aosa,Aet,Ajr,Vuosi,Kk,...,Päällyslev,Päällystlk,Päällksel,Nakos150,Nakos300,Nakos460,Runkotie,Raskos,position.lat,position.lon
0,0,6761989,1,Liikennevirasto,1.0,3.0,3.0,2.0,2014,10,...,114.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei,60.202561,24.881814
1,1,6652962,1,Liikennevirasto,1.0,3.0,8.0,1.0,2014,1,...,104.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,on,60.204074,24.880904
2,2,6840693,1,Liikennevirasto,1.0,3.0,8.0,1.0,2014,8,...,104.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei,60.204190,24.880878
3,3,6682862,1,Liikennevirasto,1.0,3.0,21.0,2.0,2014,3,...,114.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei,60.202459,24.881586
4,4,6646346,1,Liikennevirasto,1.0,3.0,30.0,2.0,2014,11,...,115.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,on,60.202429,24.881425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28340,28340,6924514,3,Muu,NaN,NaN,NaN,NaN,2014,8,...,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,on,60.228066,25.064978
28341,28341,6924567,3,Muu,NaN,NaN,NaN,NaN,2014,7,...,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei,60.861603,26.667586
28342,28342,6924574,3,Muu,NaN,NaN,NaN,NaN,2014,8,...,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei,61.492871,23.496168
28343,28343,6924769,3,Muu,NaN,NaN,NaN,NaN,2014,7,...,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei,60.401336,22.366238


#### A.5 Create new dataframe row id value column for the source file

In [18]:
input_a = df_as.values.tolist()
input_b = df_a.values.tolist()
output = []

def getinputbindex(iter_a):
    for iter_b in input_b[::-1]:
        if iter_a[0] >= iter_b[0]:
            return [iter_b[0],iter_b[cn]]

for iter_a in input_a:
    input_b_value = getinputbindex(iter_a)

    output.append([iter_a[0], input_b_value[0], input_b_value[1]])

In [19]:
# Create output to dataframe and Rename column names
df_res_1 = pd.DataFrame(output)
df_res_1 = df_res_1.set_axis(['index_val', 'hndl_idx_val', handle_frame_id], axis=1, inplace=False)

In [20]:
df_res_1

,index_val,hndl_idx_val,alue_id
0,0,0,10191
1,1,0,10191
2,2,0,10191
3,3,0,10191
4,4,0,10191
...,...,...,...
28340,28340,27457,204413
28341,28341,27457,204413
28342,28342,27457,204413
28343,28343,27457,204413


#### A.6 Create new dataframe with new id (key) value (without dropped columns of the first and second  files)

In [21]:
#After joining drop column from the target data frame (column are required for the join).
df1 = df_as.merge(df_res_1[[handle_frame_id, 'index_val']], how = 'left',
                left_on = 'index_val', right_on = 'index_val').drop(columns = ['index_val'])

In [22]:
df1

,Onnett_id,Tienpit,Tienpitsel,Tie,Aosa,Aet,Ajr,Vuosi,Kk,Päivä,...,Päällystlk,Päällksel,Nakos150,Nakos300,Nakos460,Runkotie,Raskos,position.lat,position.lon,alue_id
0,6761989,1,Liikennevirasto,1.0,3.0,3.0,2.0,2014,10,30.10.2014,...,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei,60.202561,24.881814,10191
1,6652962,1,Liikennevirasto,1.0,3.0,8.0,1.0,2014,1,07.01.2014,...,10,kovat asfaltit,97.0,76.0,57.0,1.0,on,60.204074,24.880904,10191
2,6840693,1,Liikennevirasto,1.0,3.0,8.0,1.0,2014,8,29.08.2014,...,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei,60.204190,24.880878,10191
3,6682862,1,Liikennevirasto,1.0,3.0,21.0,2.0,2014,3,15.03.2014,...,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei,60.202459,24.881586,10191
4,6646346,1,Liikennevirasto,1.0,3.0,30.0,2.0,2014,11,15.11.2014,...,10,kovat asfaltit,97.0,76.0,57.0,1.0,on,60.202429,24.881425,10191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28340,6924514,3,Muu,NaN,NaN,NaN,NaN,2014,8,07.08.2014,...,-1,Ei arvoa,NaN,NaN,NaN,NaN,on,60.228066,25.064978,204413
28341,6924567,3,Muu,NaN,NaN,NaN,NaN,2014,7,25.07.2014,...,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei,60.861603,26.667586,204413
28342,6924574,3,Muu,NaN,NaN,NaN,NaN,2014,8,06.08.2014,...,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei,61.492871,23.496168,204413
28343,6924769,3,Muu,NaN,NaN,NaN,NaN,2014,7,26.07.2014,...,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei,60.401336,22.366238,204413


In [23]:
# Change column order, new created id value (from dropped rows id's) next to main id column
cols = df1.columns.tolist()
cols.insert(1, cols.pop(-1))
df1 = df1[cols]

In [24]:
df1

,Onnett_id,alue_id,Tienpit,Tienpitsel,Tie,Aosa,Aet,Ajr,Vuosi,Kk,...,Päällyslev,Päällystlk,Päällksel,Nakos150,Nakos300,Nakos460,Runkotie,Raskos,position.lat,position.lon
0,6761989,10191,1,Liikennevirasto,1.0,3.0,3.0,2.0,2014,10,...,114.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei,60.202561,24.881814
1,6652962,10191,1,Liikennevirasto,1.0,3.0,8.0,1.0,2014,1,...,104.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,on,60.204074,24.880904
2,6840693,10191,1,Liikennevirasto,1.0,3.0,8.0,1.0,2014,8,...,104.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei,60.204190,24.880878
3,6682862,10191,1,Liikennevirasto,1.0,3.0,21.0,2.0,2014,3,...,114.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei,60.202459,24.881586
4,6646346,10191,1,Liikennevirasto,1.0,3.0,30.0,2.0,2014,11,...,115.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,on,60.202429,24.881425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28340,6924514,204413,3,Muu,NaN,NaN,NaN,NaN,2014,8,...,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,on,60.228066,25.064978
28341,6924567,204413,3,Muu,NaN,NaN,NaN,NaN,2014,7,...,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei,60.861603,26.667586
28342,6924574,204413,3,Muu,NaN,NaN,NaN,NaN,2014,8,...,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei,61.492871,23.496168
28343,6924769,204413,3,Muu,NaN,NaN,NaN,NaN,2014,7,...,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei,60.401336,22.366238


In [25]:
# Create new dataframe, without second file dropped columns
df1 = df1.drop(columns=hndl2_onnett_cols)

In [26]:
df1

,Onnett_id,alue_id,Vuosi,Kk,Päivä,Kuolleet,Loukkaant,Vakavuusko,Vakavuus,Tunti,...,Valoisuus,Valsel,Sää,Sääsel,Onnpaikka,Onnpaiksel,Lämpötila,Katuosoite,position.lat,position.lon
0,6761989,10191,2014,10,30.10.2014,0,0,0,Ei henkilövahinkoja,2,...,4,tie valaistu,1,kirkas,1,ajorata,10.0,VT1 X HUOPALAHDENTIE,60.202561,24.881814
1,6652962,10191,2014,1,07.01.2014,0,0,0,Ei henkilövahinkoja,13,...,1,päivänvalo,4,vesisade,1,ajorata,2.0,HUOPALAHDENTIE X TURUNVÄYLÄ,60.204074,24.880904
2,6840693,10191,2014,8,29.08.2014,0,0,0,Ei henkilövahinkoja,8,...,1,päivänvalo,2,pilvipouta,1,ajorata,16.0,TIE 1 X PITÄJÄNMÄENTIE,60.204190,24.880878
3,6682862,10191,2014,3,15.03.2014,0,0,0,Ei henkilövahinkoja,14,...,1,päivänvalo,6,räntäsade,1,ajorata,4.0,HUOPALAHDENTIE X TURUNVÄYLÄ,60.202459,24.881586
4,6646346,10191,2014,11,15.11.2014,0,0,0,Ei henkilövahinkoja,19,...,4,tie valaistu,2,pilvipouta,1,ajorata,2.0,HUOPALAHDENTIE X TURUNVÄYLÄ,60.202429,24.881425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28340,6924514,204413,2014,8,07.08.2014,0,0,0,Ei henkilövahinkoja,12,...,1,päivänvalo,1,kirkas,4,pysäköintialue tai piha,30.0,MYLLYRENGINTIE 2,60.228066,25.064978
28341,6924567,204413,2014,7,25.07.2014,0,0,0,Ei henkilövahinkoja,15,...,1,päivänvalo,2,pilvipouta,1,ajorata,31.0,ALAKYLÄNTIE X LÄNTINEN RATATIE,60.861603,26.667586
28342,6924574,204413,2014,8,06.08.2014,0,0,0,Ei henkilövahinkoja,17,...,1,päivänvalo,1,kirkas,1,ajorata,31.0,AARONINKATU X LÄHDEKORVENTIE,61.492871,23.496168
28343,6924769,204413,2014,7,26.07.2014,0,2,2,Loukkaantumiseen johtanut,15,...,1,päivänvalo,1,kirkas,4,pysäköintialue tai piha,32.0,ISÄNNÄNKATU 5,60.401336,22.366238


## B. Editing second result file

#### B.1 Split way type columns to new dataframe

In [27]:
df_b = pd.DataFrame(df[hndl2_file_cols])
df_b = df_b.drop_duplicates()

In [28]:
df_b

,Vuosi,Tienpit,Tienpitsel,Tie,Aosa,Aet,Ajr,Taajmerk,Taajamasel,Liikvalot,...,Poikkileik,Poikleikse,Päällyslev,Päällystlk,Päällksel,Nakos150,Nakos300,Nakos460,Runkotie,Raskos
0,2014,1,Liikennevirasto,1.0,3.0,3.0,2.0,0,E,3,...,2,2 ajor,114.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei
1,2014,1,Liikennevirasto,1.0,3.0,8.0,1.0,1,K,-1,...,2,2 ajor,104.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,on
2,2014,1,Liikennevirasto,1.0,3.0,8.0,1.0,1,K,-1,...,2,2 ajor,104.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei
3,2014,1,Liikennevirasto,1.0,3.0,21.0,2.0,0,E,1,...,2,2 ajor,114.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei
4,2014,1,Liikennevirasto,1.0,3.0,30.0,2.0,1,K,1,...,2,2 ajor,115.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,on
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28126,2014,3,Muu,NaN,NaN,NaN,NaN,1,K,-1,...,-1,Ei arvoa,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei
28162,2014,3,Muu,NaN,NaN,NaN,NaN,0,E,-1,...,-1,Ei arvoa,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,Ei arvoa
28222,2014,3,Muu,NaN,NaN,NaN,NaN,-1,Ei arvoa,-1,...,-1,Ei arvoa,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei
28309,2014,3,Muu,NaN,NaN,NaN,NaN,0,E,-1,...,-1,Ei arvoa,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei


#### B.2 Create new id value to new handle2 dataframe

In [29]:
df_b[handle2_frame_id] = df_b.reset_index().index.astype(int)
df_b.set_index(df_b.columns[-1], inplace=True)
df_b.reset_index(inplace=True)
df_b

,tieomin_id,Vuosi,Tienpit,Tienpitsel,Tie,Aosa,Aet,Ajr,Taajmerk,Taajamasel,...,Poikkileik,Poikleikse,Päällyslev,Päällystlk,Päällksel,Nakos150,Nakos300,Nakos460,Runkotie,Raskos
0,0,2014,1,Liikennevirasto,1.0,3.0,3.0,2.0,0,E,...,2,2 ajor,114.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei
1,1,2014,1,Liikennevirasto,1.0,3.0,8.0,1.0,1,K,...,2,2 ajor,104.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,on
2,2,2014,1,Liikennevirasto,1.0,3.0,8.0,1.0,1,K,...,2,2 ajor,104.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei
3,3,2014,1,Liikennevirasto,1.0,3.0,21.0,2.0,0,E,...,2,2 ajor,114.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei
4,4,2014,1,Liikennevirasto,1.0,3.0,30.0,2.0,1,K,...,2,2 ajor,115.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,on
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17871,17871,2014,3,Muu,NaN,NaN,NaN,NaN,1,K,...,-1,Ei arvoa,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei
17872,17872,2014,3,Muu,NaN,NaN,NaN,NaN,0,E,...,-1,Ei arvoa,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,Ei arvoa
17873,17873,2014,3,Muu,NaN,NaN,NaN,NaN,-1,Ei arvoa,...,-1,Ei arvoa,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei
17874,17874,2014,3,Muu,NaN,NaN,NaN,NaN,0,E,...,-1,Ei arvoa,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei


#### B.3 Create new column with values from old row index value to new handle2 dataframe

In [30]:
# Place new index value column to first
df_b = df_b.rename_axis('index_val').reset_index()

In [31]:
idx = df_b.index
df_b.loc[idx]

,index_val,tieomin_id,Vuosi,Tienpit,Tienpitsel,Tie,Aosa,Aet,Ajr,Taajmerk,...,Poikkileik,Poikleikse,Päällyslev,Päällystlk,Päällksel,Nakos150,Nakos300,Nakos460,Runkotie,Raskos
0,0,0,2014,1,Liikennevirasto,1.0,3.0,3.0,2.0,0,...,2,2 ajor,114.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei
1,1,1,2014,1,Liikennevirasto,1.0,3.0,8.0,1.0,1,...,2,2 ajor,104.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,on
2,2,2,2014,1,Liikennevirasto,1.0,3.0,8.0,1.0,1,...,2,2 ajor,104.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei
3,3,3,2014,1,Liikennevirasto,1.0,3.0,21.0,2.0,0,...,2,2 ajor,114.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei
4,4,4,2014,1,Liikennevirasto,1.0,3.0,30.0,2.0,1,...,2,2 ajor,115.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,on
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17871,17871,17871,2014,3,Muu,NaN,NaN,NaN,NaN,1,...,-1,Ei arvoa,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei
17872,17872,17872,2014,3,Muu,NaN,NaN,NaN,NaN,0,...,-1,Ei arvoa,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,Ei arvoa
17873,17873,17873,2014,3,Muu,NaN,NaN,NaN,NaN,-1,...,-1,Ei arvoa,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei
17874,17874,17874,2014,3,Muu,NaN,NaN,NaN,NaN,0,...,-1,Ei arvoa,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei


In [32]:
# Get column place number of new id value
cn2 = df_b.columns.get_loc(handle2_frame_id)
cn2

1

#### B.4 Create new column with values from old row index value to new handle2 dataframe

In [33]:
# Create new dataframe without dropped columns
df_bs = df.drop(columns=hndl2_file_cols)

In [34]:
df_bs

,Onnett_id,Kk,Päivä,Kuolleet,Loukkaant,Vakavuusko,Vakavuus,ELY,Elynimi,Poliisipri,...,Onnpaikka,Onnpaiksel,Maakunta,Maakuntsel,Kunta,Kuntasel,Lämpötila,Katuosoite,position.lat,position.lon
0,6761989,10,30.10.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,0,...,1,ajorata,1,Uusimaa,91,Helsinki,10.0,VT1 X HUOPALAHDENTIE,60.202561,24.881814
1,6652962,1,07.01.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,0,...,1,ajorata,1,Uusimaa,91,Helsinki,2.0,HUOPALAHDENTIE X TURUNVÄYLÄ,60.204074,24.880904
2,6840693,8,29.08.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,0,...,1,ajorata,1,Uusimaa,91,Helsinki,16.0,TIE 1 X PITÄJÄNMÄENTIE,60.204190,24.880878
3,6682862,3,15.03.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,0,...,1,ajorata,1,Uusimaa,91,Helsinki,4.0,HUOPALAHDENTIE X TURUNVÄYLÄ,60.202459,24.881586
4,6646346,11,15.11.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,0,...,1,ajorata,1,Uusimaa,91,Helsinki,2.0,HUOPALAHDENTIE X TURUNVÄYLÄ,60.202429,24.881425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28340,6924514,8,07.08.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,0,...,4,pysäköintialue tai piha,1,Uusimaa,91,Helsinki,30.0,MYLLYRENGINTIE 2,60.228066,25.064978
28341,6924567,7,25.07.2014,0,0,0,Ei henkilövahinkoja,3,Kaakkois-Suomi,0,...,1,ajorata,8,Kymenlaakso,286,Kouvola,31.0,ALAKYLÄNTIE X LÄNTINEN RATATIE,60.861603,26.667586
28342,6924574,8,06.08.2014,0,0,0,Ei henkilövahinkoja,4,Pirkanmaa,0,...,1,ajorata,6,Pirkanmaa,536,Nokia,31.0,AARONINKATU X LÄHDEKORVENTIE,61.492871,23.496168
28343,6924769,7,26.07.2014,0,2,2,Loukkaantumiseen johtanut,2,Varsinais-Suomi,0,...,4,pysäköintialue tai piha,2,Varsinais-Suomi,202,Kaarina,32.0,ISÄNNÄNKATU 5,60.401336,22.366238


In [35]:
# Place new index value column to first
df_bs = df_bs.rename_axis('index_val').reset_index()

In [36]:
idx = df_bs.index
df_bs.loc[idx]

,index_val,Onnett_id,Kk,Päivä,Kuolleet,Loukkaant,Vakavuusko,Vakavuus,ELY,Elynimi,...,Onnpaikka,Onnpaiksel,Maakunta,Maakuntsel,Kunta,Kuntasel,Lämpötila,Katuosoite,position.lat,position.lon
0,0,6761989,10,30.10.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,...,1,ajorata,1,Uusimaa,91,Helsinki,10.0,VT1 X HUOPALAHDENTIE,60.202561,24.881814
1,1,6652962,1,07.01.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,...,1,ajorata,1,Uusimaa,91,Helsinki,2.0,HUOPALAHDENTIE X TURUNVÄYLÄ,60.204074,24.880904
2,2,6840693,8,29.08.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,...,1,ajorata,1,Uusimaa,91,Helsinki,16.0,TIE 1 X PITÄJÄNMÄENTIE,60.204190,24.880878
3,3,6682862,3,15.03.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,...,1,ajorata,1,Uusimaa,91,Helsinki,4.0,HUOPALAHDENTIE X TURUNVÄYLÄ,60.202459,24.881586
4,4,6646346,11,15.11.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,...,1,ajorata,1,Uusimaa,91,Helsinki,2.0,HUOPALAHDENTIE X TURUNVÄYLÄ,60.202429,24.881425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28340,28340,6924514,8,07.08.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,...,4,pysäköintialue tai piha,1,Uusimaa,91,Helsinki,30.0,MYLLYRENGINTIE 2,60.228066,25.064978
28341,28341,6924567,7,25.07.2014,0,0,0,Ei henkilövahinkoja,3,Kaakkois-Suomi,...,1,ajorata,8,Kymenlaakso,286,Kouvola,31.0,ALAKYLÄNTIE X LÄNTINEN RATATIE,60.861603,26.667586
28342,28342,6924574,8,06.08.2014,0,0,0,Ei henkilövahinkoja,4,Pirkanmaa,...,1,ajorata,6,Pirkanmaa,536,Nokia,31.0,AARONINKATU X LÄHDEKORVENTIE,61.492871,23.496168
28343,28343,6924769,7,26.07.2014,0,2,2,Loukkaantumiseen johtanut,2,Varsinais-Suomi,...,4,pysäköintialue tai piha,2,Varsinais-Suomi,202,Kaarina,32.0,ISÄNNÄNKATU 5,60.401336,22.366238


#### B.5 Create second new dataframe row id value column for the source file

In [37]:
input_a = df_bs.values.tolist()
input_b = df_b.values.tolist()
output2 = []

def getinputbindex(iter_a):
    for iter_b in input_b[::-1]:
        if iter_a[0] >= iter_b[0]:
            return [iter_b[0],iter_b[cn2]]

for iter_a in input_a:
    input_b_value = getinputbindex(iter_a)

    output2.append([iter_a[0], input_b_value[0], input_b_value[1]])

In [38]:
# Create output to dataframe and Rename column names
df_res_2 = pd.DataFrame(output2)
df_res_2 = df_res_2.set_axis(['index_val', 'hndl_idx_val', handle2_frame_id], axis=1, inplace=False)

In [39]:
df_res_2

,index_val,hndl_idx_val,tieomin_id
0,0,0,0
1,1,1,1
2,2,2,2
3,3,3,3
4,4,4,4
...,...,...,...
28340,28340,17875,17875
28341,28341,17875,17875
28342,28342,17875,17875
28343,28343,17875,17875


#### B.6 Create second new dataframe with new id (key) value (without dropped columns of the first and second  files)

In [40]:
#After joining drop column from the target data frame (column are required for the join).
df2 = df_bs.merge(df_res_2[[handle2_frame_id, 'index_val']], how = 'left',
                left_on = 'index_val', right_on = 'index_val').drop(columns = ['index_val'])

In [41]:
df2

,Onnett_id,Kk,Päivä,Kuolleet,Loukkaant,Vakavuusko,Vakavuus,ELY,Elynimi,Poliisipri,...,Onnpaiksel,Maakunta,Maakuntsel,Kunta,Kuntasel,Lämpötila,Katuosoite,position.lat,position.lon,tieomin_id
0,6761989,10,30.10.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,0,...,ajorata,1,Uusimaa,91,Helsinki,10.0,VT1 X HUOPALAHDENTIE,60.202561,24.881814,0
1,6652962,1,07.01.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,0,...,ajorata,1,Uusimaa,91,Helsinki,2.0,HUOPALAHDENTIE X TURUNVÄYLÄ,60.204074,24.880904,1
2,6840693,8,29.08.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,0,...,ajorata,1,Uusimaa,91,Helsinki,16.0,TIE 1 X PITÄJÄNMÄENTIE,60.204190,24.880878,2
3,6682862,3,15.03.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,0,...,ajorata,1,Uusimaa,91,Helsinki,4.0,HUOPALAHDENTIE X TURUNVÄYLÄ,60.202459,24.881586,3
4,6646346,11,15.11.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,0,...,ajorata,1,Uusimaa,91,Helsinki,2.0,HUOPALAHDENTIE X TURUNVÄYLÄ,60.202429,24.881425,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28340,6924514,8,07.08.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,0,...,pysäköintialue tai piha,1,Uusimaa,91,Helsinki,30.0,MYLLYRENGINTIE 2,60.228066,25.064978,17875
28341,6924567,7,25.07.2014,0,0,0,Ei henkilövahinkoja,3,Kaakkois-Suomi,0,...,ajorata,8,Kymenlaakso,286,Kouvola,31.0,ALAKYLÄNTIE X LÄNTINEN RATATIE,60.861603,26.667586,17875
28342,6924574,8,06.08.2014,0,0,0,Ei henkilövahinkoja,4,Pirkanmaa,0,...,ajorata,6,Pirkanmaa,536,Nokia,31.0,AARONINKATU X LÄHDEKORVENTIE,61.492871,23.496168,17875
28343,6924769,7,26.07.2014,0,2,2,Loukkaantumiseen johtanut,2,Varsinais-Suomi,0,...,pysäköintialue tai piha,2,Varsinais-Suomi,202,Kaarina,32.0,ISÄNNÄNKATU 5,60.401336,22.366238,17875


In [42]:
# Change column order, new created id value (from dropped rows id's) next to main id column
cols = df2.columns.tolist()
cols.insert(1, cols.pop(-1))
df2 = df2[cols]

In [43]:
df2

,Onnett_id,tieomin_id,Kk,Päivä,Kuolleet,Loukkaant,Vakavuusko,Vakavuus,ELY,Elynimi,...,Onnpaikka,Onnpaiksel,Maakunta,Maakuntsel,Kunta,Kuntasel,Lämpötila,Katuosoite,position.lat,position.lon
0,6761989,0,10,30.10.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,...,1,ajorata,1,Uusimaa,91,Helsinki,10.0,VT1 X HUOPALAHDENTIE,60.202561,24.881814
1,6652962,1,1,07.01.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,...,1,ajorata,1,Uusimaa,91,Helsinki,2.0,HUOPALAHDENTIE X TURUNVÄYLÄ,60.204074,24.880904
2,6840693,2,8,29.08.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,...,1,ajorata,1,Uusimaa,91,Helsinki,16.0,TIE 1 X PITÄJÄNMÄENTIE,60.204190,24.880878
3,6682862,3,3,15.03.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,...,1,ajorata,1,Uusimaa,91,Helsinki,4.0,HUOPALAHDENTIE X TURUNVÄYLÄ,60.202459,24.881586
4,6646346,4,11,15.11.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,...,1,ajorata,1,Uusimaa,91,Helsinki,2.0,HUOPALAHDENTIE X TURUNVÄYLÄ,60.202429,24.881425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28340,6924514,17875,8,07.08.2014,0,0,0,Ei henkilövahinkoja,1,Uusimaa,...,4,pysäköintialue tai piha,1,Uusimaa,91,Helsinki,30.0,MYLLYRENGINTIE 2,60.228066,25.064978
28341,6924567,17875,7,25.07.2014,0,0,0,Ei henkilövahinkoja,3,Kaakkois-Suomi,...,1,ajorata,8,Kymenlaakso,286,Kouvola,31.0,ALAKYLÄNTIE X LÄNTINEN RATATIE,60.861603,26.667586
28342,6924574,17875,8,06.08.2014,0,0,0,Ei henkilövahinkoja,4,Pirkanmaa,...,1,ajorata,6,Pirkanmaa,536,Nokia,31.0,AARONINKATU X LÄHDEKORVENTIE,61.492871,23.496168
28343,6924769,17875,7,26.07.2014,0,2,2,Loukkaantumiseen johtanut,2,Varsinais-Suomi,...,4,pysäköintialue tai piha,2,Varsinais-Suomi,202,Kaarina,32.0,ISÄNNÄNKATU 5,60.401336,22.366238


In [44]:
# Create new dataframe, without first file dropped columns
df2 = df2.drop(columns=hndl_file_cols)

In [45]:
df2

,Onnett_id,tieomin_id,Kk,Päivä,Kuolleet,Loukkaant,Vakavuusko,Vakavuus,Tunti,Vkpv,...,Valoisuus,Valsel,Sää,Sääsel,Onnpaikka,Onnpaiksel,Lämpötila,Katuosoite,position.lat,position.lon
0,6761989,0,10,30.10.2014,0,0,0,Ei henkilövahinkoja,2,Torstai,...,4,tie valaistu,1,kirkas,1,ajorata,10.0,VT1 X HUOPALAHDENTIE,60.202561,24.881814
1,6652962,1,1,07.01.2014,0,0,0,Ei henkilövahinkoja,13,Tiistai,...,1,päivänvalo,4,vesisade,1,ajorata,2.0,HUOPALAHDENTIE X TURUNVÄYLÄ,60.204074,24.880904
2,6840693,2,8,29.08.2014,0,0,0,Ei henkilövahinkoja,8,Perjantai,...,1,päivänvalo,2,pilvipouta,1,ajorata,16.0,TIE 1 X PITÄJÄNMÄENTIE,60.204190,24.880878
3,6682862,3,3,15.03.2014,0,0,0,Ei henkilövahinkoja,14,Lauantai,...,1,päivänvalo,6,räntäsade,1,ajorata,4.0,HUOPALAHDENTIE X TURUNVÄYLÄ,60.202459,24.881586
4,6646346,4,11,15.11.2014,0,0,0,Ei henkilövahinkoja,19,Lauantai,...,4,tie valaistu,2,pilvipouta,1,ajorata,2.0,HUOPALAHDENTIE X TURUNVÄYLÄ,60.202429,24.881425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28340,6924514,17875,8,07.08.2014,0,0,0,Ei henkilövahinkoja,12,Torstai,...,1,päivänvalo,1,kirkas,4,pysäköintialue tai piha,30.0,MYLLYRENGINTIE 2,60.228066,25.064978
28341,6924567,17875,7,25.07.2014,0,0,0,Ei henkilövahinkoja,15,Perjantai,...,1,päivänvalo,2,pilvipouta,1,ajorata,31.0,ALAKYLÄNTIE X LÄNTINEN RATATIE,60.861603,26.667586
28342,6924574,17875,8,06.08.2014,0,0,0,Ei henkilövahinkoja,17,Keskiviikko,...,1,päivänvalo,1,kirkas,1,ajorata,31.0,AARONINKATU X LÄHDEKORVENTIE,61.492871,23.496168
28343,6924769,17875,7,26.07.2014,0,2,2,Loukkaantumiseen johtanut,15,Lauantai,...,1,päivänvalo,1,kirkas,4,pysäköintialue tai piha,32.0,ISÄNNÄNKATU 5,60.401336,22.366238


# C. Target and Result files

#### C.1 Create Target file from first and second new dataframes

In [46]:
# Joining dataframes df1 and df2 (with both new id value columns and without dropped/handled columns)
df_t = df1.merge(df2[['Onnett_id','tieomin_id']], how = 'left', left_on = 'Onnett_id', right_on = 'Onnett_id')

In [47]:
# Change column order, new joined id value (from second file) next to two id columns
cols = df_t.columns.tolist()
cols.insert(2, cols.pop(-1))
df_t = df_t[cols]

In [48]:
df_t

,Onnett_id,alue_id,tieomin_id,Vuosi,Kk,Päivä,Kuolleet,Loukkaant,Vakavuusko,Vakavuus,...,Valoisuus,Valsel,Sää,Sääsel,Onnpaikka,Onnpaiksel,Lämpötila,Katuosoite,position.lat,position.lon
0,6761989,10191,0,2014,10,30.10.2014,0,0,0,Ei henkilövahinkoja,...,4,tie valaistu,1,kirkas,1,ajorata,10.0,VT1 X HUOPALAHDENTIE,60.202561,24.881814
1,6652962,10191,1,2014,1,07.01.2014,0,0,0,Ei henkilövahinkoja,...,1,päivänvalo,4,vesisade,1,ajorata,2.0,HUOPALAHDENTIE X TURUNVÄYLÄ,60.204074,24.880904
2,6840693,10191,2,2014,8,29.08.2014,0,0,0,Ei henkilövahinkoja,...,1,päivänvalo,2,pilvipouta,1,ajorata,16.0,TIE 1 X PITÄJÄNMÄENTIE,60.204190,24.880878
3,6682862,10191,3,2014,3,15.03.2014,0,0,0,Ei henkilövahinkoja,...,1,päivänvalo,6,räntäsade,1,ajorata,4.0,HUOPALAHDENTIE X TURUNVÄYLÄ,60.202459,24.881586
4,6646346,10191,4,2014,11,15.11.2014,0,0,0,Ei henkilövahinkoja,...,4,tie valaistu,2,pilvipouta,1,ajorata,2.0,HUOPALAHDENTIE X TURUNVÄYLÄ,60.202429,24.881425
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28340,6924514,204413,17875,2014,8,07.08.2014,0,0,0,Ei henkilövahinkoja,...,1,päivänvalo,1,kirkas,4,pysäköintialue tai piha,30.0,MYLLYRENGINTIE 2,60.228066,25.064978
28341,6924567,204413,17875,2014,7,25.07.2014,0,0,0,Ei henkilövahinkoja,...,1,päivänvalo,2,pilvipouta,1,ajorata,31.0,ALAKYLÄNTIE X LÄNTINEN RATATIE,60.861603,26.667586
28342,6924574,204413,17875,2014,8,06.08.2014,0,0,0,Ei henkilövahinkoja,...,1,päivänvalo,1,kirkas,1,ajorata,31.0,AARONINKATU X LÄHDEKORVENTIE,61.492871,23.496168
28343,6924769,204413,17875,2014,7,26.07.2014,0,2,2,Loukkaantumiseen johtanut,...,1,päivänvalo,1,kirkas,4,pysäköintialue tai piha,32.0,ISÄNNÄNKATU 5,60.401336,22.366238


In [49]:
# Create target csv file and save it
df_t.to_csv(target_file, sep=rslt_delimiter, encoding=rslt_encoding, index=False)

#### C.2 Create Result file from first handled dataframes

In [50]:
# Check and make a needed corrections to df_a
df_a

,index_val,ELY,Elynimi,Poliisipri,Piirinimi,Maakunta,Maakuntsel,Kunta,Kuntasel,alue_id
0,0,1,Uusimaa,0,Ei arvoa,1,Uusimaa,91,Helsinki,10191
1,16,1,Uusimaa,0,Ei arvoa,1,Uusimaa,49,Espoo,10149
2,44,1,Uusimaa,0,Ei arvoa,1,Uusimaa,257,Kirkkonummi,101257
3,54,1,Uusimaa,0,Ei arvoa,1,Uusimaa,927,Vihti,101927
4,71,1,Uusimaa,0,Ei arvoa,1,Uusimaa,444,Lohja,101444
...,...,...,...,...,...,...,...,...,...,...
308,18927,4,Pirkanmaa,0,Ei arvoa,6,Pirkanmaa,604,Pirkkala,406604
309,19142,10,Etelä-Pohjanmaa,0,Ei arvoa,14,Etelä-Pohjanmaa,164,Jalasjärvi,10014164
310,22422,2,Varsinais-Suomi,0,Ei arvoa,4,Satakunta,319,Köyliö,204319
311,27392,1,Uusimaa,0,Ei arvoa,7,Päijät-Häme,283,Hämeenkoski,107283


In [51]:
# Change column order, new created id value to first
cols = df_a.columns.tolist()
cols.insert(0, cols.pop(-1))
df_a = df_a[cols]
# Drop column from the result dataframe (column are required for the join)
df_a = df_a.drop(columns='index_val')

In [52]:
df_a

,alue_id,ELY,Elynimi,Poliisipri,Piirinimi,Maakunta,Maakuntsel,Kunta,Kuntasel
0,10191,1,Uusimaa,0,Ei arvoa,1,Uusimaa,91,Helsinki
1,10149,1,Uusimaa,0,Ei arvoa,1,Uusimaa,49,Espoo
2,101257,1,Uusimaa,0,Ei arvoa,1,Uusimaa,257,Kirkkonummi
3,101927,1,Uusimaa,0,Ei arvoa,1,Uusimaa,927,Vihti
4,101444,1,Uusimaa,0,Ei arvoa,1,Uusimaa,444,Lohja
...,...,...,...,...,...,...,...,...,...
308,406604,4,Pirkanmaa,0,Ei arvoa,6,Pirkanmaa,604,Pirkkala
309,10014164,10,Etelä-Pohjanmaa,0,Ei arvoa,14,Etelä-Pohjanmaa,164,Jalasjärvi
310,204319,2,Varsinais-Suomi,0,Ei arvoa,4,Satakunta,319,Köyliö
311,107283,1,Uusimaa,0,Ei arvoa,7,Päijät-Häme,283,Hämeenkoski


In [53]:
# Create result csv file and save it
df_a.to_csv(result_file, sep=rslt_delimiter, encoding=rslt_encoding, index=False)

#### C.3 Create Result2 file from second handled dataframes

In [54]:
# Check and make a needed corrections to df_b
df_b

,index_val,tieomin_id,Vuosi,Tienpit,Tienpitsel,Tie,Aosa,Aet,Ajr,Taajmerk,...,Poikkileik,Poikleikse,Päällyslev,Päällystlk,Päällksel,Nakos150,Nakos300,Nakos460,Runkotie,Raskos
0,0,0,2014,1,Liikennevirasto,1.0,3.0,3.0,2.0,0,...,2,2 ajor,114.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei
1,1,1,2014,1,Liikennevirasto,1.0,3.0,8.0,1.0,1,...,2,2 ajor,104.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,on
2,2,2,2014,1,Liikennevirasto,1.0,3.0,8.0,1.0,1,...,2,2 ajor,104.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei
3,3,3,2014,1,Liikennevirasto,1.0,3.0,21.0,2.0,0,...,2,2 ajor,114.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei
4,4,4,2014,1,Liikennevirasto,1.0,3.0,30.0,2.0,1,...,2,2 ajor,115.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,on
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17871,17871,17871,2014,3,Muu,NaN,NaN,NaN,NaN,1,...,-1,Ei arvoa,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei
17872,17872,17872,2014,3,Muu,NaN,NaN,NaN,NaN,0,...,-1,Ei arvoa,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,Ei arvoa
17873,17873,17873,2014,3,Muu,NaN,NaN,NaN,NaN,-1,...,-1,Ei arvoa,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei
17874,17874,17874,2014,3,Muu,NaN,NaN,NaN,NaN,0,...,-1,Ei arvoa,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei


In [55]:
# Drop column from the result dataframe (column are required for the join)
df_b = df_b.drop(columns='index_val')

In [56]:
df_b

,tieomin_id,Vuosi,Tienpit,Tienpitsel,Tie,Aosa,Aet,Ajr,Taajmerk,Taajamasel,...,Poikkileik,Poikleikse,Päällyslev,Päällystlk,Päällksel,Nakos150,Nakos300,Nakos460,Runkotie,Raskos
0,0,2014,1,Liikennevirasto,1.0,3.0,3.0,2.0,0,E,...,2,2 ajor,114.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei
1,1,2014,1,Liikennevirasto,1.0,3.0,8.0,1.0,1,K,...,2,2 ajor,104.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,on
2,2,2014,1,Liikennevirasto,1.0,3.0,8.0,1.0,1,K,...,2,2 ajor,104.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei
3,3,2014,1,Liikennevirasto,1.0,3.0,21.0,2.0,0,E,...,2,2 ajor,114.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,ei
4,4,2014,1,Liikennevirasto,1.0,3.0,30.0,2.0,1,K,...,2,2 ajor,115.0,10,kovat asfaltit,97.0,76.0,57.0,1.0,on
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17871,17871,2014,3,Muu,NaN,NaN,NaN,NaN,1,K,...,-1,Ei arvoa,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei
17872,17872,2014,3,Muu,NaN,NaN,NaN,NaN,0,E,...,-1,Ei arvoa,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,Ei arvoa
17873,17873,2014,3,Muu,NaN,NaN,NaN,NaN,-1,Ei arvoa,...,-1,Ei arvoa,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei
17874,17874,2014,3,Muu,NaN,NaN,NaN,NaN,0,E,...,-1,Ei arvoa,NaN,-1,Ei arvoa,NaN,NaN,NaN,NaN,ei


In [57]:
# Create result csv file and save it
df_b.to_csv(result2_file, sep=rslt_delimiter, encoding=rslt_encoding, index=False)